# Colab'a hoş geldiniz.

<div class="markdown-google-sans">

## Diğer Kaynaklar

### Colab'de Not Defterleriyle Çalışma

</div>

- [Colab'e genel bakış](/notebooks/basic_features_overview.ipynb)
- [Markdown rehberi](/notebooks/markdown_guide.ipynb)
- [Kitaplıkları içe aktarma ve bağımlıları yükleme](/notebooks/snippets/importing_libraries.ipynb)
- [GitHub'da not defteri kaydetme ve yükleme](https://colab.research.google.com/github/googlecolab/colabtools/blob/main/notebooks/colab-github-demo.ipynb)
- [Etkileşimli formlar](/notebooks/forms.ipynb)
- [Etkileşimli widget'lar](/notebooks/widgets.ipynb)

<div class="markdown-google-sans">

<a name="working-with-data"></a>
### Verilerle Çalışma
</div>

- [Veri yükleme: Drive, E-Tablolar ve Google Cloud Storage](/notebooks/io.ipynb)
- [Grafikler: Veri görselleştirme](/notebooks/charts.ipynb)
- [BigQuery'yi kullanmaya başlama](/notebooks/bigquery.ipynb)

<div class="markdown-google-sans">

### Makine Öğrenimi

<div>

Google'ın online Makine Öğrenimi kursu da dahil olmak üzere Makine Öğrenimi ile ilgili birkaç not defterini burada bulabilirsiniz. Daha fazlası için <a href="https://developers.google.com/machine-learning/crash-course/">tam kurs web sitesine</a> bakın.
- [Pandas DataFrame'e giriş](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/pandas_dataframe_ultraquick_tutorial.ipynb)
- [Pandas'ı hızlandırmak için RAPIDS cuDF'ye giriş](https://nvda.ws/rapids-cudf)
- [cuML'nin hızlandırıcı modunu kullanmaya başlama](https://colab.research.google.com/github/rapidsai-community/showcase/blob/main/getting_started_tutorials/cuml_sklearn_colab_demo.ipynb)
- [Sentetik veriler kullanarak tf.keras ile doğrusal regresyon](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/linear_regression_with_synthetic_data.ipynb)

<div class="markdown-google-sans">

<a name="using-accelerated-hardware"></a>
### Hızlandırılmış Donanım Kullanma
</div>

- [GPU'lar ile TensorFlow](/notebooks/gpu.ipynb)
- [Colab'deki TPU'lar](/notebooks/tpu.ipynb)

<div class="markdown-google-sans">

<a name="machine-learning-examples"></a>

### Öne çıkan örnekler

</div>

- <a href="https://tensorflow.org/hub/tutorials/tf2_image_retraining">Bir Resim Sınıflandırıcıyı Yeniden Eğitme</a>: Çiçekleri ayırt etmek için önceden eğitilmiş bir resim sınıflandırıcının üzerine bir Keras modeli inşa eder.
- <a href="https://tensorflow.org/hub/tutorials/tf2_text_classification">Metin Sınıflandırma</a>: IMDB'deki film yorumlarını <em>olumlu</em> veya <em>olumsuz</em> olarak sınıflandırır.
- <a href="https://tensorflow.org/hub/tutorials/tf2_arbitrary_image_stylization">Stil Aktarımı</a>: Resimler arasında stil aktarımı yapmak için derin öğrenmeyi kullanır.
- <a href="https://tensorflow.org/hub/tutorials/retrieval_with_tf_hub_universal_encoder_qa">Çok Dilli Evrensel Cümle Kodlayıcı Soru-Cevap</a>: SQuAD veri kümesinden soruları cevaplamak için bir makine öğrenimi modeli kullanır.
- <a href="https://tensorflow.org/hub/tutorials/tweening_conv3d">Video İnterpolasyonu</a>: Bir videonun ilk ve son karesi arasında ne olduğunu tahmin eder.
